In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow import keras
import os
import cv2
import numpy as np

In [ ]:
# allows all images to be displayed at once (else only displays the last call to plt.imshow())
# https://stackoverflow.com/a/41210974
def displayImage(image, caption = None, colour = None) -> None:
    plt.figure()
    if(colour != None):
        plt.imshow(image, cmap=colour)
    else:
        plt.imshow(image)
        
    if(caption != None):
        # display caption below picture (https://stackoverflow.com/a/51486361)
        plt.figtext(0.5, 0.01, caption, wrap=True, horizontalalignment='center', fontsize=12)

## Predict on unlabelled data
* using similar data (from same camera-trap dataset) that is unlabelled but semi-distinct from the train/test data
* TODO: use camera trap images from Internet

### Load the model

In [ ]:
model = keras.models.load_model('saved_models/ZFNet-Gaussian-Blur-and-Canny-no-empy-plus-hist-eq-dark-images.h5')

### Load up test images (no pre-processing)

In [ ]:
img_fns = next(os.walk('data/unlabelled_test_data'), (None, None, []))[2]  # [] if no file
img_fns.sort()
print('img_fns: {}'.format(img_fns))

unlabelled_test_images = []
desired_size = 224
for fn in img_fns:
    im = cv2.imread('data/unlabelled_test_data/{}'.format(fn))
    
    # resize the image
    old_size = im.shape[:2]
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    im = cv2.resize(im, (new_size[1], new_size[0]))

    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)

    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
    
    unlabelled_test_images.append(np.asarray(new_im))
    
unlabelled_test_images = np.stack(unlabelled_test_images, axis = 0)
print('unlabelled_test_images shape: {}'.format(unlabelled_test_images.shape))

In [ ]:
# y_prob = model.predict(unlabelled_test_images)
y_pred = (model.predict(unlabelled_test_images) > 0.5).astype("int32")
print('y_pred:\n{}'.format(y_pred))
y_classes = y_pred.argmax(axis=-1)
print('y_classes: {}'.format(y_classes))

In [ ]:
# sorted_test_classes = sorted(test_classes)

In [ ]:
# https://stackoverflow.com/a/57516809
# test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.)
# test_generator = test_datagen('data',
#                               # only read images from `unlabelled_test_data` directory
#                               classes=['unlabelled_test_data'],
#                               # don't generate labels
#                               class_mode=None,
#                               # don't shuffle
#                               shuffle=False,
#                               # use same size as in training
#                               target_size=(240, 240))
# preds = model.predict_generator(test_generator)

# paths = next(os.walk('data/unlabelled_test_data'), (None, None, []))[2]  # [] if no file
# paths.sort()
# print('paths: {}'.format(paths))
# a_img_rand = np.random.randint(0,len(paths))
# img = keras.utils.load_img('data/unlabelled_test_data/{}'.format(paths[a_img_rand]))
# displayImage(img, 'Random loaded image')
# img = keras.utils.img_to_array(img)
# classes = ['Empty photo', 'Kangaroo', 'Human Presense/Deployment', 'Emu', 'Cat', 'Fox', 'Bird', 'Rabbit', 'Dog', 'Other']
# print('classes: {}'.format(classes))
# class_list = list(classes)
# print('class_list: {}'.format(class_list))
# pred_i = y_prob[a_img_rand]
# print('pred_i: {}'.format(pred_i))
# pred = class_list[pred_i]
# print('pred: {}'.format(pred))
# displayImage(img, 'Predicted class: '.format(pred))